In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import pandas as pd

with open("/content/drive/MyDrive/comment_classification_project/ClaimBuster_Datasets/datasets/2xNCS.json", "r") as f: #change directory
    data2 = json.load(f)

df2 = pd.DataFrame(data2)

print(df2)

from datasets import Dataset, ClassLabel
df2 = Dataset.from_pandas(df2)

if "__index_level_0__" in df2.column_names:
    df2 = df2.remove_columns(["__index_level_0__"])

class_label = ClassLabel(names=["opinion", "claim"])

df2 = df2.cast_column("label", class_label)

print(df2)
# 0 = opinion
# 1 = claim

      sentence_id  label                                               text
0           15083      1  When I made my decision to stop all trade with...
1           16799      1  We've got the highest inflation we've had in t...
2           32570      1  They started from that little area, and now th...
3           17644      1  Yes, there has been an increase in poverty, bu...
4           32512      1  And, yes, when I was a senator, I did vote to ...
...           ...    ...                                                ...
8287        23355      0  I guess you'd say I'm a good steward of the land.
8288        23298      0  Now, either he's going to break all these wond...
8289        23042      0  Wanda Blackmore I met here from Missouri, the ...
8290         8512      0  The third is medical care for the aged which i...
8291        25642      0                         We have met young America.

[8292 rows x 3 columns]


Casting the dataset:   0%|          | 0/8292 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence_id', 'label', 'text'],
    num_rows: 8292
})


In [ ]:
!pip install transformers datasets scikit-learn

In [ ]:
!pip install numpy==1.26.4

In [ ]:
from datasets import Dataset

split_dataset = df2.train_test_split(
    test_size=0.2,
    seed=42,
    stratify_by_column="label"
)

train_ds2 = split_dataset["train"]
test_ds2 = split_dataset["test"]


print("Train shape:", train_ds2.shape)
print("Test shape:", test_ds2.shape)

print(train_ds2)
print(test_ds2)

Train shape: (6633, 3)
Test shape: (1659, 3)
Dataset({
    features: ['sentence_id', 'label', 'text'],
    num_rows: 6633
})
Dataset({
    features: ['sentence_id', 'label', 'text'],
    num_rows: 1659
})


In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import AutoTokenizer

model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.save_pretrained("./results")

def preprocess(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_ds2 = train_ds2.map(preprocess, batched=True)


/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/spm.model',
 './results/added_tokens.json',
 './results/tokenizer.json')

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch

print("CUDA available?", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0))

CUDA available? True
Device count: 1
Device name: Tesla T4


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds2,
)

trainer.train()

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: caijun-ng (caijun-ng-singapore-management-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.660200
20,0.632300
30,0.572200
40,0.482500
50,0.468000
60,0.431800
70,0.354900
80,0.519200
90,0.600000
100,0.502200


TrainOutput(global_step=2490, training_loss=0.2074159440098248, metrics={'train_runtime': 839.0488, 'train_samples_per_second': 23.716, 'train_steps_per_second': 2.968, 'total_flos': 1308935196467712.0, 'train_loss': 0.2074159440098248, 'epoch': 3.0})

In [ ]:
# trained model has been saved under Backend -> debertav3base

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

def preprocess(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_test2 = test_ds2.map(preprocess, batched=True)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/1659 [00:00<?, ? examples/s]

In [ ]:
import os

print(os.listdir("/content/results"))

['checkpoint-1660', 'checkpoint-2490', 'checkpoint-830', 'runs']


In [ ]:
# Load trained model
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/results/checkpoint-2490", # trained model has been saved under Backend -> debertav3base. Change path accordingly
    num_labels=2
)


In [ ]:
import numpy as np
trainer = Trainer(model=model)

predictions = trainer.predict(tokenized_test2)

logits = predictions.predictions

y_pred = np.argmax(logits, axis=1)
y_true = predictions.label_ids

print("Predictions:", y_pred)
print("Ground Truth:", y_true)


Predictions: [0 0 1 ... 0 0 1]
Ground Truth: [0 0 1 ... 0 0 1]


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

precision, recall, f1, _ = precision_recall_fscore_support(
    y_true,
    y_pred,
    average="binary"
)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Accuracy: 0.9312839059674503
Precision: 0.8927
Recall: 0.9024
F1-score: 0.8975
